In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import joblib

In [2]:
# Load the JSON file
with open('test.json', 'r') as file:
    json_data = json.load(file)

In [12]:
# Define possible variations for each attribute
model_nbr_variations = ['ModelNbr', 'model', 'modelno', 'model no', 'MODEL', 'MODEL NO']
manufacturer_variations = ['Manufacturer', 'manufacturer', 'make']
serial_no_variations = ['SerialNo', 'serial', 'serialnumber','serialno']

In [13]:
# Extract attributes
model_nbr = None
manufacturer = None
serial_no = None

In [14]:
for variation in model_nbr_variations:
    if variation in json_data:
        model_nbr = json_data[variation]
        break

for variation in manufacturer_variations:
    if variation in json_data:
        manufacturer = json_data[variation]
        break

for variation in serial_no_variations:
    if variation in json_data:
        serial_no = json_data[variation]
        break

In [15]:
X = pd.DataFrame({
        'SerialNo': [serial_no],
        'Manufacturer': [manufacturer],
        'ModelNbr': [model_nbr]
    })

In [16]:
X

,SerialNo,Manufacturer,ModelNbr
0,6631302324,PANASONIC,CS-RE12MKR


In [17]:
special_values = [1234, 'UNKOWN', 'Unknown', 'NA', 'NULL', 'UNKNOWN', 'TBA', 'N/A', 'NOT VISIBLE', '123TEST', 'UNABLE TO LOCATE', 'NO ID', 'NO ACCESS', 'UNKOWN', 'NaN', 'na', 'AS PER PICS','nan']

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
X_copy = X.copy()

# Replace values using .loc to avoid SettingWithCopyWarning
X_copy.loc[:, 'SerialNo'] = X_copy['SerialNo'].replace(special_values, pd.NA)
X_copy.loc[:, 'Manufacturer'] = X_copy['Manufacturer'].replace(special_values, pd.NA)
X_copy.loc[:, 'ModelNbr'] = X_copy['ModelNbr'].replace(special_values, pd.NA)

X = X_copy

In [18]:
LE_SerialNo = joblib.load('model/LE_Serial')
LE_ModelNbr = joblib.load('model/LE_ModelNbr')
LE_Manufact = joblib.load('model/LE_Manufact')
LE_Asset = joblib.load('model/LE_Asset')

In [19]:
X['ModelNbr'] = LE_ModelNbr.transform(X['ModelNbr'].astype(str))
X['SerialNo'] = LE_SerialNo.transform(X['SerialNo'].astype(str))
X['Manufacturer'] = LE_Manufact.transform(X['Manufacturer'].astype(str))

In [20]:
X

,SerialNo,Manufacturer,ModelNbr
0,58560,781,19284


In [21]:
rf_classifier = joblib.load('model/rf_final2')

In [23]:
y_pred_encoded = rf_classifier.predict(X)

In [24]:
y_pred_encoded

array([540])

In [25]:
y_actual = LE_Asset.inverse_transform(y_pred_encoded)

In [26]:
y_actual

array(['WALL MOUNTED SPLIT SYSTEM'], dtype=object)

In [27]:
json_data['PredictedAssetType'] = y_actual[0]

In [28]:
with open('test.json', 'w') as output_file:
        json.dump(json_data, output_file)